# W266 Final Project - Load_DataCleansing_TextProcessing

In [14]:
# Initial data load, summary, and high level sanity checks

# Import a bunch of libraries.
import glob
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from nltk.corpus import brown
nltk.download('brown')
from nltk.corpus import webtext
#import parseyface

reviews = pd.read_csv('RR_Pared_All_Active_2018-07-09.csv', encoding='utf-8')


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mfrazzini/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package brown to /home/mfrazzini/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [15]:
print(reviews.shape)
print(reviews.iloc[0])

(369888, 8)
MODELID                13032
TESTIMONIALID          95423
TESTIMONIAL      very useful
OVERALL                  NaN
ORGANIZATION              10
APPEARANCE                10
PRICE_VALUE                9
DURABILITY               NaN
Name: 0, dtype: object


In [16]:
#Remove html special characters that end up in the review text
from bs4 import BeautifulSoup 


In [17]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(brown.words())

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))


In [18]:
print(correction("Looking"))

Looking


In [ ]:
def text_process(text):
    #Takes in a string of text, then performs the following:
    #1. Remove all html special characters, with regex to pull partially remove html special characters
    #2. Spelling correction
    #3. Remove all punctuation
    #4. Remove all stopwords
    #5. Stemming of words
    #6. Return the cleaned text as a list of words
    if type(text) is str: #if len(text) > 0:
        nopunc = [char for char in text if char not in string.punctuation]
        #print(nopunc)
        nopunc = ''.join(nopunc)
        review = []
        for word in nopunc.split():
            if word.lower() not in stopwords.words('english'):
                #html_decoded = BeautifulSoup(word, "lxml")
                #html_decoded_string = html_decoded.string
                if re.search('\w(\d+)\w', word): #html_decoded_string):
                    html_decoded_string = word.replace("39","'") #html_decoded_string.replace("39","'")
                spell_corrected = correction(word)
                if word != spell_corrected:
                    spell_corrections.append(spell_corrected + ':' + word)
                #porter = PorterStemmer()
                #stemmed = porter.stem(spell_corrected)
                review.append(spell_corrected) #review.append(stemmed)
        return ' '.join(review)
    else:
        return ''

spell_corrections = []    
    
for index, row in reviews.iterrows():
    print("Currently on record: {}; Currently iterrated {}% of records".format(index, (index + 1)/len(reviews.index) * 100),
          end="\r")
    reviews.at[index, 'TESTIMONIAL'] = text_process(row['TESTIMONIAL'])

word_vectorizer = CountVectorizer(ngram_range=(3,6), analyzer='word', stop_words=None)
sparse_matrix = word_vectorizer.fit_transform(data_group['TESTIMONIAL'])
frequencies = sum(sparse_matrix).data

result_group = pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['frequency'])
result_sort_group = result_group.sort_values('frequency', ascending=False)
result_sort_group.to_csv("Baseline_ngram_output_w_spellcorrect.csv")



In [14]:
import csv

correction_counts = Counter(spell_corrections)
sorted_corrections = sorted(correction_counts.items(), key=lambda kv: kv[1])

with open('spell_corrections_output.csv','wb') as f:
    w = csv.writer(f)
    w.writerows(sorted_corrections.items())

AttributeError: 'list' object has no attribute 'items'

In [15]:
print(sorted_corrections)








[('Reliability:Readability', 1), ('functioned:functionned', 1), ('opportunity:oppurtunity', 1), ('concealed:concealer', 1), ('0:00', 1), ('Luckily:Lunckily', 1), ('gagline:naglene', 1), ('months:monthes', 1), ('businessmen:businesswomen', 1), ('with:mith', 1), ('Also:ALso', 1), ('learners:lerners', 1), ('that:phat', 1), ('onto:onetoo', 1), ('wish:jewish', 1), ('tales:talis', 1), ('recipients:recipants', 1), ('liberty:lifetry', 1), ('groves:gromets', 1), ('markedly:markely', 1), ('fit:fit59', 1), ('ooze:oozes', 1), ('supplements:supplements59', 1), ('life:eife', 1), ('designer:Designer', 1), ('standard:stansard', 1), ('Hansen:Hanson', 1), ('spinning:espinning', 1), ('formability:foldability', 1), ('mistaking:mispacking', 1), ('cold:coldI', 1), ('total:topall', 1), ('efficient:efficientI', 1), ('clothes:myclothes', 1), ('snowstorm:snowstorms', 1), ('capita:Narita', 1), ('734:1734W', 1), ('124:1234H', 1), ('434:434D', 1), ('freaks:freaks34', 1), ('It:1It', 1), ('My:2My', 1), ('trips:trips